In [94]:
#import packages etc
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy
import re
from elasticsearch import Elasticsearch
matplotlib.rc('font', **{'size': 15})
from elasticsearch.helpers import scan
import time
import operator

In [95]:
#declare variables
print("Declaring variables.")

#epoch times of start and end of time interval. 
#first week of february. monday 30. Jan to sunday 5. feb.
#found using epochconverter.com
#epoch time must be in milliseconds
#epoch_start = 1488668400000 #Mar 5th 00.00.00 (week 10)
epoch_start =1485730800000 #jan 30th 00.00.00
#epoch_end = 1489273199000 #mar 11th 23.59.59
epoch_end = 1486335599000 #feb 5th 23.59.59
#epoch_end = 1485820799000 # jan30th 23.59.59 (one day for testing)
#epoch_end = 1485920799000

loop_time = epoch_start #time variable in loop

#loop time increments . one minute? five minutes?
increment = (5 * 60 * 1000)
timevector = [epoch_start]
dummy = epoch_start
while dummy <= epoch_end:
    dummy += increment
    timevector.append(dummy)


#thresholds for defining an error
failure_rate = .9 #failure rate threshold. Q: what is the threshold of red cells in DDM dashboard?
file_thresh = 10 #number of files threshold.
persist_thresh = 45 * 60 * 1000; #milliseconds. time needed before alert is sent.

#number of alarms
alarms = 0

#date and time format of
#ex: 2017-03-27 10:03:10
pattern = '%Y-%m-%d %H:%M:%S'



#some kind of dictionary with pairs of sites, and some variable keeping track of time
#if pairs stay on this list too long an alarm is raised {[source,destination]:interval}
observed_endpoints = {}

#dict of when an alert was made and what pair is failing {when: [source,receiver]}
print("Done.")

Declaring variables.
Done.


In [96]:
#Elasticsearch querries
print("Defining ElasticSearch querries.")
query_done = {
    "size": 0,
    "query": {
    "bool": {
    "must": [
    {
        "range": {
        "@timestamp": {
            "gte": epoch_start,
            "lte": epoch_end,
            "format": "epoch_millis"
            }
        }
    },
        {"term": {"event_type": "transfer-done"}},
        {"term": {"payload.scope": "data16_13TeV"}}
    ]
    }
    }
}
query_failed = {
    "size": 0,
    "query": {
    "bool": {
    "must": [
    {
        "range": {
            "@timestamp": {
            "gte": epoch_start,
            "lte": epoch_end,
            "format": "epoch_millis"
             }
            }
           },
          {"term": {"event_type": "transfer-failed"}},
          {"term": {"payload.scope": "data16_13TeV"}}
        ]
        }
    }
}


#make scrolls
print("Defining scrolls.")
# make scrolls
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60) 
my_index = "rucio-events-2017*"
scroll_done = scan(es, query=query_done, index=my_index, scroll='5m', timeout="5m", size=100)
scroll_errors = scan(es, query=query_failed, index=my_index, scroll='5m', timeout="5m", size=100)


print("Done.")

Defining ElasticSearch querries.
Defining scrolls.
Done.


In [97]:
#could be useful to track how much time is spent on this block
time_tracker = time.time()

#nested dict. every entry is at the start of an interval.
#interval[time][pair] is number of transfers between pairs in given interval
interval_transfers = {}
interval_failures = {}
interval_ratios = {}
for interval in timevector:
    interval_transfers[interval] = {}
    interval_failures[interval] = {}
    interval_ratios[interval] = {}

alarm_endpoints = {}

print('Starting Sorting Loop Transfers')
for entry in scroll_done:
    source = entry['_source']['payload']['src-rse']
    destination = entry['_source']['payload']['dst-rse']
    pair = (source,destination)
    submitted_at = entry['_source']['payload']['submitted_at'] 
    #convert submitted at to epoch time
    #must be in milliseconds
    submitted_at = int(time.mktime(time.strptime(submitted_at, pattern)))*1000  

    for interval in timevector:
        if interval <= submitted_at and submitted_at <= interval+increment:
            if pair in interval_transfers[interval].keys():
                interval_transfers[interval][pair] = interval_transfers[interval][pair] + 1
            if pair not in interval_transfers[interval].keys():
                interval_transfers[interval][pair] = 1
    
print('Sorting Loop Transfers Done')
print('Starting Sorting Loop Failures')
for entry in scroll_errors:
    source = entry['_source']['payload']['src-rse']
    destination = entry['_source']['payload']['dst-rse']
    pair = (source,destination)
    submitted_at = entry['_source']['payload']['submitted_at'] 
    #convert submitted at to epoch time
    #must be in milliseconds
    submitted_at = int(time.mktime(time.strptime(submitted_at, pattern)))*1000  

    for interval in timevector:
        if interval <= submitted_at and submitted_at <= interval+increment:
            if pair in interval_failures[interval].keys():
                interval_failures[interval][pair] = interval_failures[interval][pair] + 1
            if pair not in interval_failures[interval].keys():
                interval_failures[interval][pair] = 1

print('Sorting Loop Transfers Done')

print('Calculating Failure Ratios')
#calculating the file transfer failure rate betwen pairs of grid sites in each time interval
for interval in interval_transfers:
    for pair in interval_transfers[interval].keys():
        if pair in interval_failures[interval].keys():
            interval_ratios[interval][pair] = float(interval_failures[interval][pair])/(float(interval_failures[interval][pair])+float(interval_transfers[interval][pair]))
        else:
            interval_ratios[interval][pair] = 0.0
    for pair in interval_failures[interval].keys():
        if pair not in interval_transfers[interval].keys():
            interval_ratios[interval][pair] = 1.0
    
    
#used for debugging
#for interval in interval_ratios:
#    for pair in interval_ratios[interval].keys():
#        if interval_ratios[interval][pair] > 1.0:
#            print('pair: ',pair,'rate: ', interval_ratios[interval][pair])




print('Failure Ratios Calulated')

print('Starting Alarm Loop')

alarm_list = []

#observed_endpoints
for interval in interval_ratios:
    remove = []
    for pair in interval_ratios[interval].keys():
        #number of total failed and successful transfer betwen pair
        transfers = 0
        if pair in interval_transfers[interval].keys():
            transfers += interval_transfers[interval][pair]
        if pair in interval_failures[interval].keys():
            transfers += interval_failures[interval][pair]
        #failure rate must be over threshold and numer of transfers over threshold
        if(interval_ratios[interval][pair] >= failure_rate and transfers >= file_thresh):
            if pair not in observed_endpoints.keys():
                observed_endpoints[pair] = interval
            if pair in observed_endpoints.keys() and interval - observed_endpoints[pair] >= persist_thresh:
                if pair in alarm_endpoints.keys():
                    alarm_endpoints[pair] += 1
                if pair not in alarm_endpoints.keys():
                    alarm_endpoints[pair] = 1
                alarms += 1
                alarm_list.append([interval,pair])
                del observed_endpoints[pair]
                
    # if not in ratios or threshold too low, remove from watchlist
        if(pair in observed_endpoints.keys() and (interval_ratios[interval][pair] <= failure_rate or transfers < file_thresh)):
            
            del observed_endpoints[pair]
            
                        
print('Alarm Loop Done')

print("time spent: ", (time.time() - time_tracker)/60, "minutes.")  
print('Alarms: ', alarms)

Starting Sorting Loop Transfers
Sorting Loop Transfers Done
Starting Sorting Loop Failures
Sorting Loop Transfers Done
Calculating Failure Ratios
Failure Ratios Calulated
Starting Alarm Loop
Alarm Loop Done
time spent:  36.59641253153483 minutes.
Alarms:  172


In [98]:
#study the endpoint pairs that send alarms

#print(alarm_endpoints)
import collections

most_mentioned = {}
for entry in alarm_endpoints.keys():
    for word in entry:
        if word in most_mentioned.keys():
            most_mentioned[word] += alarm_endpoints[entry]
        if word not in most_mentioned.keys():
            most_mentioned[word] = alarm_endpoints[entry]

sorted_most_mentioned = collections.OrderedDict(sorted(most_mentioned.items()))
            
for entry in most_mentioned.keys():
    print('site: ',entry,'alarms: ',most_mentioned[entry])
            
print('')

#time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(1347517370))
for entry in alarm_list:
    print('When: ',time.strftime(pattern,time.localtime(entry[0]/1000)),', pair: ',entry[1])


site:  CSCS-LCG2_DATADISK alarms:  69
site:  JINR-LCG2_DATADISK alarms:  11
site:  AGLT2_DATADISK alarms:  7
site:  IN2P3-CPPM_DATADISK alarms:  14
site:  DESY-ZN_DATADISK alarms:  4
site:  UKI-SCOTGRID-GLASGOW_DATADISK alarms:  45
site:  CERN-PROD_DATADISK alarms:  24
site:  UKI-SOUTHGRID-OX-HEP_DATADISK alarms:  17
site:  CERN-PROD_DERIVED alarms:  7
site:  GRIF-LPNHE_DATADISK alarms:  25
site:  PRAGUELCG2_DATADISK alarms:  25
site:  TOKYO-LCG2_DATADISK alarms:  1
site:  UKI-SCOTGRID-DURHAM_DATADISK alarms:  19
site:  DESY-HH_DATADISK alarms:  5
site:  UKI-SOUTHGRID-SUSX_DATADISK alarms:  2
site:  RRC-KI_DATADISK alarms:  1
site:  UKI-NORTHGRID-SHEF-HEP_DATADISK alarms:  3
site:  SWT2_CPB_DATADISK alarms:  1
site:  UKI-SOUTHGRID-BHAM-HEP_DATADISK alarms:  3
site:  NET2_DATADISK alarms:  11
site:  UNIBE-LHEP_DATADISK alarms:  2
site:  MWT2_DATADISK alarms:  5
site:  RO-14-ITIM_DATADISK alarms:  1
site:  GRIF-LAL_DATADISK alarms:  25
site:  INFN-COSENZA_DATADISK alarms:  10
site:  CERN